# General data reader for AQUA 
## Streaming mode

The reader includes the ability to simulate a data streaming, that is to retrieve chunks of data of specific time length.

In [1]:
import sys
sys.path.append("../..")  # hack to import module -- to be removed later
from aqua import Reader, catalogue

Let's load some IFS data. We first instantiate a `Reader` object specifying the type of data which we want to read from the catalogue. 

In [2]:
reader = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d")

Then the actual data are read with the `retrieve` method. By default, the streaming mode is deactivated (`streaming = False`) and the `retrieve` method will return all the requested data. If you want to activate the streaming mode you should specify the argument `streaming=True`.

There are also three optional parameters: `stream_step`, `stream_unit` and `stream_startdate`: 

`stream_unit` is a string representing the unit of time to stream the data by (e.g. 'hours', 'days', 'months') (None)

`stream_step` is an integer representing the number of time steps to stream the data by.  Default = 1

`stream_startdate` is a string representing the starting date for streaming the data (e.g. '2020-02-25'). If it is not specified, the function will use the first date in the dataset.

Let's say we want to stream the data every three days. We need to call:

In [3]:
data = reader.retrieve(streaming=True, stream_step=3, stream_unit='days')
data

tp: corrected multiplying by density of water 1000 kg m-3
tp: corrected dividing by accumulation time 10800 s


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

We correctly retrieved data from 2020-01-20 to 2020-01-23, keeping the original hourly temporal resolution. 

If `retrieve` is called a second time on the same input data and with the same parameters, it will return the subsequent chunk of 3 days data. This is because the method keeps track of the state of the streaming process through the use of internal attributes.

In [4]:
data = reader.retrieve(streaming=True, stream_step=3, stream_unit='days')
data

<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-01-23 ... 2020-01-25T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

By calling `retrieve` a second time, we correctly retrieved data from 2020-01-23 to 2020-01-26.

If the retrieve method in streaming mode is called multiple times with the same parameters, it will return the data in chunks until all of the data has been streamed. The function will automatically determine the appropriate start and end points for each chunk based on the internal state of the streaming process.

If we want to reset the state of the streaming process we can call the `reset_stream` method. This means that if `retrieve` in streaming mode is called again after calling `reset_stream`, it will start streaming the input data from the beginning.

In [5]:
reader.reset_stream()
data = reader.retrieve(streaming=True, stream_step=3, stream_unit='days')
data

<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

We got back to the first three days, 2020-01-20 to 2020-01-23.

By default, the data stream will start from the first date of the input file.
If we want it to start from a specific date, we can use the parameter `stream_startdate`:

In [6]:
reader.reset_stream()
data = reader.retrieve(streaming=True, stream_step=1, stream_unit='months', stream_startdate = '2020-05-01')
data

<xarray.Dataset>
Dimensions:  (time: 744, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-31T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

the `stream_startdate` is fixed the first time you call `retrieve()`. If you call `retrieve()` a second time, data will continue to be retrieved from the last date of the first `retrieve()` call. If you want to start again from a different starting date, you shoud call `reset_stream()` first and then call `retrieve()` specifying the new `stream_startdate`. 

All the streaming parameters can also be specified at the moment of the reader initialization.
If we call `retrieve` without specifying any option or parameter, the ones in the reader initialization will be taken.

In [16]:
reader2 = Reader(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", streaming=True, stream_step=3,  stream_unit='days', stream_startdate = '2020-05-01')
data = reader2.retrieve()
data

tp: corrected multiplying by density of water 1000 kg m-3
tp: corrected dividing by accumulation time 10800 s


<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-03T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In this case, if `stream_startdate` is specified in the reader initialization, a call to `reset_stream()` will not make the stream start again from the the first date of the input file, but from `stream_startdate`.

In [17]:
reader2.reset_stream()
data = reader2.retrieve()
data

<xarray.Dataset>
Dimensions:  (time: 72, value: 26306560, level: 5)
Coordinates:
  * level    (level) int64 1 2 3 4 5
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-03T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

There's a second way to use the streaming mode. 
If we don't specify the `unit` parameter, the data is streamed by `stream_step` steps of the original time resolution of input data.
If we call:

In [18]:
reader.reset_stream()
data = reader.retrieve(streaming=True, stream_step=10)
data

tp: corrected multiplying by density of water 1000 kg m-3
tp: corrected dividing by accumulation time 10800 s


<xarray.Dataset>
Dimensions:  (time: 10, value: 26306560, level: 5)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-20T09:00:00
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

We get the first 10 hours data.
Again, if we want the streaming to start from a specific date, we can use the parameter `stream_startdate`:

In [19]:
reader.reset_stream()
data = reader.retrieve(streaming=True, stream_step=10, stream_startdate = '2020-05-01' )
data

<xarray.Dataset>
Dimensions:  (time: 10, value: 26306560, level: 5)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 1 2 3 4 5
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-05-01 ... 2020-05-01T09:00:00
Dimensions without coordinates: value
Data variables: (12/81)
    100si    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100u     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    100v     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10u      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    10v      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    2d       (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    ...       ...
    uvb      (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vike     (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipie    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vipile   (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    vithe    (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
    z        (time, value) float64 dask.array<chunksize=(1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

The streaming mode of the `retrieve` method relies on the `stream` method of the class `Reader`. 
If needed, this method can be used independently on a given xarray.Dataset, for example on a subset of the input data:

In [20]:
reader.reset_stream()
data = reader.retrieve()
reader.stream(data["pr"], stream_step=3, stream_unit='days' )

<xarray.DataArray 'pr' (time: 72, value: 26306560)>
dask.array<where, shape=(72, 26306560), dtype=float64, chunksize=(1, 26306560), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 ... 2020-01-22T23:00:00
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
Dimensions without coordinates: value
Attributes: (12/20)
    paramId:                    228
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                surface
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid
    target_units:               kg m-2 s-1
    factor:                     1.0
    offset:                     0
    src_units:                  kg m-2 s-1
    units_fixed:                1

Another possibility to deal with data streaming is to call the `stream_generator` method of the class `Reader`.
This can be done from the `retrieve` method through the argument `streaming_generator = True`.

In [21]:
reader.reset_stream()
data_gen = reader.retrieve(streaming_generator=True, stream_step=3, stream_unit = 'months')
data_gen

<generator object Reader.stream_generator at 0x7f2c98e02b20>

`data_gen` is now a generator object that yields the requested 3 month-chunks of data. We can do operations with them by iterating on the generator object:

In [22]:
for data in data_gen:
    print(f"start_date: {data.time[0].values} stop_date: {data.time[-1].values}")

start_date: 2020-01-20T00:00:00.000000000 stop_date: 2020-04-19T23:00:00.000000000
start_date: 2020-04-20T00:00:00.000000000 stop_date: 2020-07-19T23:00:00.000000000
start_date: 2020-07-20T00:00:00.000000000 stop_date: 2020-10-19T23:00:00.000000000
start_date: 2020-10-20T00:00:00.000000000 stop_date: 2021-01-01T00:00:00.000000000
